In [1]:
import pandas
import numpy

import torch
import torch.nn as nn
import torch.nn.functional as F

import json

In [2]:
file_path = '../data/moral_data_original.json'

# Load the JSON file into a Python dictionary
with open(file_path, 'r') as f:
    data = json.load(f)

# Now 'data' is a Python dictionary containing the JSON structure

In [8]:
print(data['moral_dialogue']['12']['ALLEN'][1])

 Can I use my real name on the radio? 


Autoencoder (H - Basic version)

In [2]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim=768, latent_dim=20):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z


Autoencoder (H - VAE version)

In [3]:
class VAE(nn.Module):
    def __init__(self, input_dim=768, latent_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc_mu = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)

        self.fc_decode1 = nn.Linear(latent_dim, 64)
        self.fc_decode2 = nn.Linear(64, 256)
        self.fc_out = nn.Linear(256, input_dim)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc_decode1(z))
        h = F.relu(self.fc_decode2(h))
        return self.fc_out(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar, z


In [4]:
def vae_loss_function(recon_x, x, mu, logvar):
    recon_loss = F.mse_loss(recon_x, x, reduction='mean')
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    kl_div /= x.size(0) * x.size(1)  # normalize by batch size and dim
    return recon_loss + kl_div  

### Our main Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForMaskedLM

# === Define H: the autoencoder ===
class Autoencoder(nn.Module):
    def __init__(self, input_dim=768, latent_dim=20):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon, z

# === Load BERT ===
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert = BertForMaskedLM.from_pretrained("bert-base-uncased")
bert.eval()  # freeze for now (optional)

# === Example sentence with a masked moral word ===
sentence = "He always speaks the [MASK] truth."
inputs = tokenizer(sentence, return_tensors="pt")
mask_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]

# === Get the target label ===
target_word = "moral"
target_id = tokenizer.convert_tokens_to_ids(target_word)

# === Embed the sentence using BERT (or another encoder) ===
with torch.no_grad():
    outputs = bert.bert(**inputs)
    sentence_embedding = outputs.last_hidden_state.mean(dim=1)  # shape: [1, 768]

# === Pass through H ===
H = Autoencoder(input_dim=768, latent_dim=20)
x_recon, z = H(sentence_embedding)

# === Feed reconstructed embedding back into BERT for masked word prediction ===
# Replace the original sentence embedding with the reconstructed one
# We'll manually set BERT's last_hidden_state and run the LM head
with torch.no_grad():
    reconstructed_hidden_states = x_recon.unsqueeze(1).repeat(1, inputs.input_ids.size(1), 1)
    logits = bert.cls(reconstructed_hidden_states)  # shape: [1, seq_len, vocab_size]

# === Compute cross-entropy loss on the masked position ===
logits_masked = logits[0, mask_index, :]  # shape: [1, vocab_size]
loss = F.cross_entropy(logits_masked, torch.tensor([target_id]))

print(f"Cross-entropy loss for masked word prediction: {loss.item():.4f}")      

Load the dataset if they're already in  

In [ ]:
# Define the directory
input_dir = "/content/drive/MyDrive/Moral-inference/"

# Load each file
with open(os.path.join(input_dir, "train_data_100.json"), "r") as f:
    train_data_100 = json.load(f)

with open(os.path.join(input_dir, "train_data_200.json"), "r") as f:
    train_data_200 = json.load(f)

with open(os.path.join(input_dir, "test_data_100.json"), "r") as f:
    test_data_100 = json.load(f)

with open(os.path.join(input_dir, "test_data_200.json"), "r") as f:
    test_data_200 = json.load(f)

print("✅ Datasets loaded successfully!")
print(f"train_data_100 samples: {len(train_data_100)}")
print(f"train_data_200 samples: {len(train_data_200)}")
print(f"test_data_100 samples: {len(test_data_100)}")
print(f"test_data_200 samples: {len(test_data_200)}")

In [10]:
    # variable to control the processing of sentences
num_of_sentence_threshold = 200

# Re-import required packages after kernel reset
import json
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import os

# Load tokenizer and BERT model (frozen for now)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_model.eval()

# Re-upload the dataset file
file_path = "../data/moral_data_full.json"
if not os.path.exists(file_path):
    raise FileNotFoundError("Please re-upload the 'moral_data_full.json' file.")

# Load preprocessed JSON file
with open(file_path, "r") as f:
    moral_data = json.load(f)

moral_dialogue = moral_data["moral_dialogue"]
moral_dialogue_masked = moral_data["moral_dialogue_masked"]
ground_truths = moral_data["ground_truths"]

# Cache to store character histories and inputs for training
train_data = []

# Build training samples
for movie, characters in tqdm(moral_dialogue.items(), desc="Processing characters"):
    for character, original_sentences in characters.items():

        current_character_num_of_sentences = len(original_sentences)

        # skip if the number of sentences is below require threshold
        if current_character_num_of_sentences < num_of_sentence_threshold:
            continue

        masked_sentences = moral_dialogue_masked[movie][character]
        moral_words = ground_truths[movie][character]

        previous_embeddings = []

        # Start from the second half of the sentences
        for idx in range(len(original_sentences)//2, len(original_sentences)):
            # 1. Get masked sentence and target
            masked_sentence = masked_sentences[idx]
            target_word = moral_words[idx]

            # 2. Get all previous unmasked sentences (before current one)
            past_sentences = original_sentences[:idx]

            if not past_sentences:
                continue  # skip if no history

            # 3. Get pooled embedding of past sentences
            with torch.no_grad():
                embeddings = []
                for s in past_sentences:
                    encoded = tokenizer(s, return_tensors="pt", truncation=True, max_length=128)
                    output = bert_model(**encoded)
                    pooled = output.last_hidden_state.mean(dim=1).squeeze(0)  # [768]
                    embeddings.append(pooled)
                avg_embedding = torch.stack(embeddings, dim=0).mean(dim=0)  # [768]

            train_data.append({
                "character": character,
                "movie": movie,
                "avg_embedding": avg_embedding.tolist(),  # stored as list for now
                "masked_sentence": masked_sentence,
                "target_word": target_word
            })

# Display a few samples
import pandas as pd
import ace_tools as tools

df_preview = pd.DataFrame(train_data[:20])
tools.display_dataframe_to_user(name="Sample Training Data (Moral Prediction)", dataframe=df_preview)


Processing characters:   2%|▏         | 20/1072 [05:32<4:51:34, 16.63s/it]


KeyboardInterrupt: 